## Load modules

We use matplotlib for graphs, scipy and numpy for statistical functions and random distributions, and pyabc for inferring parameter sets. We additionally use line_profiler and memory_profiler to measure code performance.

In [1]:
import os
import tempfile

db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))

In [2]:
#import modules, which consist of python command
import copy, random, functools, pickle, math
import numpy as np
import pandas as pd
from multiprocessing import Pool
from scipy import stats
from functools import partial

In [3]:
%matplotlib inline

from pyabc import (ABCSMC,
                   RV, Distribution,
                   MedianEpsilon,
                   LocalTransition,
                   AdaptivePNormDistance)
from pyabc.visualization import plot_kde_2d
import matplotlib.pyplot as plt
import scipy as sp

/home/hz368/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/hz368/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
#load extensions
%load_ext line_profiler
%load_ext memory_profiler

In [5]:
import pickle

In [6]:
def getLineage(filename):
    #reads a tsv file of clone counts
    df = pd.read_csv(filename, sep="\t")
    counts = np.transpose(df.values).astype(float)
    header = df.columns.values
    return(header[1:].astype(float),counts[1:])

In [7]:
times, counts = getLineage("DoupeAggregate.tsv")

In [8]:
from mpmath import *
import cmath

import math
def normal_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

In [9]:
def F(x, y, t, r, gamma):
#     v = scipy.linalg.fractional_matrix_power(1 - 4 * r, 0.5)
    v = (1 - 4 * r)** 0.5    
#     w = (gamma * (1 - 2 * r) - 2 * r) / scipy.linalg.fractional_matrix_power(
#         2 * gamma * (1 - 4 * r), 0.5)
    w = (gamma * (1 - 2 * r) - 2 * r) / (2 * gamma * (1 - 4 * r)** 0.5)    
#     gg = scipy.linalg.fractional_matrix_power(1 - 4 * r, 0.5) / gamma
    gg = (1 - 4 * r)** 0.5 / gamma
    u = (1 - y) * np.exp(-gamma * t)
    u0 = (1 - y)
    Q = 1 + 2 * w - gg * u0 + (2 * r * (x - y) + y - 1) / gamma
    C = (-Q * whitm(w, 0, gg * u0) + (1 + 2 * w) * whitm(1 + w, 0, gg * u0)) / \
        (Q * whitw(w, 0, gg * u0) + 2 * whitw(1 + w, 0, gg * u0))
    f = 1 - u + \
        (u * (1 + v) - gamma * (1 + 2 * w)) / (2 * r) + \
        (gamma / (2 * r)) * ((1 + 2 * w) * whitm(1 + w, 0, u * gg) -
                             2 * C * whitw(1 + w, 0, u * gg)) / \
        (whitm(w, 0, u * gg) + C * whitw(w, 0, u * gg))
    return f


def Antal(z, t, r, gamma):
    g = []
    for k in range(len(z)):
        if z[k] == 1:
            g.append(1)
        else:
            g.append(F(z[k], z[k], t, r, gamma))
            
#     for k in range(1, len(z) + 1):
#         if z[k] == 1:
#             g[k] = 1
#         else:
#             g[k] = F(z[k], z[k], t, r, gamma)

    return g

In [10]:
def Psurv(t, l, r, rho):
    rho = rho - 0.00001
    gamma = rho / (1 - rho)
    T = l * t
    p = 1 - np.real(Antal([0], T, r, gamma))
#     if (p <= 0 or p > 1 or np.isnan(p)):
    if (p <= 0 or p > 1):
        p = Psurv(t, l, r + 0.00001, rho - 0.001)
    return p

In [11]:
def Pn(n, t, l, r, rho, N, returnConsecutive='', max_n=''):
    if max(n) >= N:
        N = max(n) + 1
    rho = rho - 0.00001
    assert (rho < 1)
    assert (rho > 0)
    gamma = rho / (1 - rho)
    T = l * t
#     k = range(N)
#     Gvals = Antal(cmath.exp(2 * np.pi * 1j * k / N), T, r, gamma)
    Gvals = Antal([cmath.exp(2 * np.pi * 1j * k / N) for k in range(int(N))], T, r, gamma)
    p = []

    if returnConsecutive:
        p = np.zeros(max_n)
#         for lp in range(1, len(n) + 1):
        for lp in range(len(n)):
#              p[n[lp]] = \
#                 np.real((1 / N) *
#                         sum(Gvals * np.exp(-2 * np.pi * 1j * k * n[lp] / N)))
            temporary = [cmath.exp(-2 * np.pi * 1j * k * n[lp] / N) for k in range(int(N))]
            p[n[lp]-1] = np.real((1 / N) *sum([a*b for a,b in zip(Gvals,temporary)]))
                               
    else:
#         for lp in range(1, len(n) + 1):
        for lp in range(len(n)):
#             p[lp] = \
#                 np.real((1 / N) * sum(Gvals *
#                                       np.exp(-2 * np.pi * 1j * k * n[lp] / N)))          
            Temporary = [cmath.exp(-2 * np.pi * 1j * k * n[lp] / N) for k in range(int(N))]
            p[lp] = np.real((1 / N) * sum(a*b for a,b in zip(Gvals,Temporary)))
#     print(Gvals)
    tst = sum(p)
    print(tst)
    if (tst > 1 or tst < 0 or np.isnan(tst)):
        print('Pathological point: r=' + str(r) + ' rho=' + str(
            rho) + '. Making 0.1% perturbation')
        p = Pn(n, t, l, r + 0.00001, rho - 0.001, N, returnConsecutive, max_n)

#     for x in range(len(p)):
#         if (p[x] < 0):
#             p[x] = 0

#     p[p < 0 & p > -1e-5] = 0

    return(p)

In [12]:
def logLikelihood(obs,lpBins,binSize):
    maxClone = len(lpBins)
    def noisyMul(a,b,i):
        #print("logP",lp, "\nobsv", obs); lp[i]*obs[i]
        print("logP",a, "\nobsv", b)
        return(a[i]*b[i])
    obsBins = [sum(obs[sum(binSize[0:n]):sum(binSize[0:(n+1)])]) for n in range(len(binSize))]
    return(sum([obsBins[j]*lpBins[j] for j in range(maxClone)]))

def allLogLike(obs,lpBins,binSize):
    return(sum([logLikelihood(o,p,binSize) for o,p in zip(obs,lpBins)]))

def logProbability(probability,binSize):
    binSizeList = [[sum(t[sum(binSize[0:n]):sum(binSize[0:(n+1)])]) for n in range(len(binSize))] for t in probability]
    logFreq = [[0 if n<=0 else math.log10(n) for n in t] for t in binSizeList]
    return(logFreq)

def calculateLikelihood(probability,binSize,data):
    lpBins = logProbability(probability, binSize)
    like = allLogLike(data,lpBins,binSize)
    #print(like)
    return(like)

In [13]:
def main(t,nRange,lambdaRange,rhoRange,rRange,binSize,data):
    maxN = max(list(map(max, nRange)))
    
#     PScanPP = np.zeros(len(lambdaRange), len(rhoRange), len(rRange),
#                        len(nRange), maxN)    
    PScanPP = [[[[np.zeros(maxN) for L4 in range(len(nRange))] for L3 in range(len(rRange))] for L2 in range(len(rhoRange))] for L1 in range(len(lambdaRange))]
    
#     PSurvScanPP = np.zeros(len(lambdaRange), len(rhoRange), len(rRange),
#                            len(nRange))    
    PSurvScanPP = [[[np.zeros(len(nRange)) for L3 in range(len(rRange))] for L2 in range(len(rhoRange))] for L1 in range(len(lambdaRange))]

    timePoints = range(len(t))
    nBadValues = 0

    for L1 in range(len(lambdaRange)):       
        PScanPP_local = [[[np.zeros(maxN) for L4 in range(len(nRange))] for L3 in range(len(rRange))] for L2 in range(len(rhoRange))]
             
        PSurvScanPP_local = [[np.zeros(len(nRange)) for L3 in range(len(rRange))] for L2 in range(len(rhoRange))]

        for L2 in range(len(rhoRange)):
            for L3 in range(len(rRange)):
                for L4 in timePoints:
                    probS = Psurv(t[L4], lambdaRange[L1], rRange[L3], rhoRange[L2])
                    avgN = 1 / probS
                    Niter = max([normal_round(int(4*avgN)), normal_round(1.5*max(nRange[L4])), 10])
                    probN = Pn(nRange[L4], t[L4], lambdaRange[L1], rRange[L3],
                               rhoRange[L2], Niter, 'returnConsecutive', maxN)

                    if any(probN > 1) or \
                            sum(probN) > 1 or \
                            probS > 1 or \
                            probS <0 or \
                            abs(probS-sum(probN))>1e-3:
                        print('Bad values at time t=' + str(t[L4]) + ' lambda=' + str(lambdaRange[L1]) + ' rho=' + str(rhoRange[L2]) + ' r=' + str(rRange[L3]))
                        print('any(probN>1=' + str(any(probN>1)) + ', sum(probN)>1=' + str(sum(probN)>1) + ', (probS-sum(probN))=' + str(probS-sum(probN)))
                        nBadValues += 1
                    else:
                        print('Completed time t=' + str(t[L4]) + ' weeks, lambda=' + str(lambdaRange[L1]) + ' rho=' + str(rhoRange[L2]) + ' r=' + str(rRange[L3]))
    
#                     PSurvScanPP_local[L2][L3][L4] = probS
                    PScanPP_local[L2][L3][L4][:] = probN/probS
                PScanPP_local[L2][L3] = calculateLikelihood(PScanPP_local[L2][L3],binSize,data)
        PSurvScanPP[L1] = PSurvScanPP_local
        PScanPP[L1] = PScanPP_local

    return(PScanPP) 

In [14]:
nSize = len(np.transpose(counts))
nBin = int(round(np.log(nSize)/np.log(2)+0.5))
binSize = np.append([1],[2**(n-1) for n in range(1,(nBin+1))])


In [15]:
times

array([  3.,  10.,  21.,  42.,  84., 180., 365.])

In [16]:
t = times/7
nRange = [list(range(1, max(np.where(t>0)[0])+1)) for t in counts]
lambdaRange = [2.9]
bins=20
rhoRange = [(i*0.8/bins) for i in range(1,bins)]
rRange = [(i*0.2/bins) for i in range(1,bins)]


In [17]:
%time result = main(times,nRange,lambdaRange,rhoRange,rRange,binSize,counts)

0.210004780388709
Bad values at time t=3.0 lambda=2.9 rho=0.04 r=0.01
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.78902814763009732')]
0.5155245881094493
Bad values at time t=10.0 lambda=2.9 rho=0.04 r=0.01
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.47174094154963064')]
0.7063848470049026
Bad values at time t=21.0 lambda=2.9 rho=0.04 r=0.01
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.22987750219969083')]
0.5700746161876148
Bad values at time t=42.0 lambda=2.9 rho=0.04 r=0.01
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.1669175818011569')]
0.30667895249418364
Bad values at time t=84.0 lambda=2.9 rho=0.04 r=0.01
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.10292371695697444')]
0.14833601575204203
Bad values at time t=180.0 lambda=2.9 rho=0.04 r=0.01
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.043196208486588783')]
0.07281126607604288
Bad values at time t=365.

0.6164368061493944
Bad values at time t=10.0 lambda=2.9 rho=0.04 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.29960015060905409')]
0.5997696381503332
Bad values at time t=21.0 lambda=2.9 rho=0.04 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.10161879130282658')]
0.2898447608333542
Bad values at time t=42.0 lambda=2.9 rho=0.04 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.041405175411937889')]
0.07608496486047564
Bad values at time t=84.0 lambda=2.9 rho=0.04 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.019135329869333889')]
0.022523529079950155
Bad values at time t=180.0 lambda=2.9 rho=0.04 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0081169302597773225')]
0.008307219465030645
Bad values at time t=365.0 lambda=2.9 rho=0.04 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0049278958413117146')]
0.3645827046700148
Bad values at 

0.5249822409108783
Bad values at time t=21.0 lambda=2.9 rho=0.04 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.071053231772869041')]
0.2159880545663456
Bad values at time t=42.0 lambda=2.9 rho=0.04 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.025982733302636257')]
0.03913986817526534
Bad values at time t=84.0 lambda=2.9 rho=0.04 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.019403934615540949')]
0.008898982816347297
Bad values at time t=180.0 lambda=2.9 rho=0.04 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.008210493641598952')]
0.0028423358208053895
Bad values at time t=365.0 lambda=2.9 rho=0.04 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0043559424621577659')]
0.4583938598101006
Bad values at time t=3.0 lambda=2.9 rho=0.04 r=0.16
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.5272822322178613')]
0.6375710313664815
Bad values at t

0.10304003401741518
Bad values at time t=84.0 lambda=2.9 rho=0.08 r=0.030000000000000006
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.038547565416386764')]
0.048137714022830395
Bad values at time t=180.0 lambda=2.9 rho=0.08 r=0.030000000000000006
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.016735152738294075')]
0.02345244842104773
Bad values at time t=365.0 lambda=2.9 rho=0.08 r=0.030000000000000006
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0082837450428738073')]
0.2744704121233825
Bad values at time t=3.0 lambda=2.9 rho=0.08 r=0.04
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.71063612946246391')]
0.5362591903490133
Bad values at time t=10.0 lambda=2.9 rho=0.08 r=0.04
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.30164913797655135')]
0.40403029147670366
Bad values at time t=21.0 lambda=2.9 rho=0.08 r=0.04
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.117155706

0.010870918520077445
Bad values at time t=180.0 lambda=2.9 rho=0.08 r=0.1
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0098119515282858883')]
0.004702467826876027
Bad values at time t=365.0 lambda=2.9 rho=0.08 r=0.1
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0051015316607717362')]
0.41367188228331353
Bad values at time t=3.0 lambda=2.9 rho=0.08 r=0.11000000000000001
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.54831705555327515')]
0.49811613657617987
Bad values at time t=10.0 lambda=2.9 rho=0.08 r=0.11000000000000001
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.18776801913271429')]
0.26354966310742534
Bad values at time t=21.0 lambda=2.9 rho=0.08 r=0.11000000000000001
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.055593838417742569')]
0.08551085159863815
Bad values at time t=42.0 lambda=2.9 rho=0.08 r=0.11000000000000001
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mp

...

0.004643449099361764
Bad values at time t=365.0 lambda=2.9 rho=0.36 r=0.11000000000000001
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0039028308168572714')]
0.34950476274900283
Bad values at time t=3.0 lambda=2.9 rho=0.36 r=0.12000000000000002
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.27444382644840132')]
0.11843064033790082
Bad values at time t=10.0 lambda=2.9 rho=0.36 r=0.12000000000000002
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.13075948021706166')]
0.06196866771525785
Bad values at time t=21.0 lambda=2.9 rho=0.36 r=0.12000000000000002
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.065566117500465954')]
0.04122111694010122
Bad values at time t=42.0 lambda=2.9 rho=0.36 r=0.12000000000000002
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.024789849450527535')]
0.012648795248764638
Bad values at time t=84.0 lambda=2.9 rho=0.36 r=0.12000000000000002
any(probN>1=False, sum(probN)>1

0.06458505778708046
Bad values at time t=10.0 lambda=2.9 rho=0.36 r=0.19
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.10977036477696067')]
0.029699078225391324
Bad values at time t=21.0 lambda=2.9 rho=0.36 r=0.19
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.055072503298911543')]
0.01922710192491509
Bad values at time t=42.0 lambda=2.9 rho=0.36 r=0.19
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.023567549583569405')]
0.0055189659333107375
Bad values at time t=84.0 lambda=2.9 rho=0.36 r=0.19
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.015982061153661253')]
0.0036407251470873462
Bad values at time t=180.0 lambda=2.9 rho=0.36 r=0.19
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0064190899012024419')]
0.001742023180917296
Bad values at time t=365.0 lambda=2.9 rho=0.36 r=0.19
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0032246897578922739')]
0.7316215073364154
Bad va

0.08926176631635134
Bad values at time t=42.0 lambda=2.9 rho=0.4 r=0.07
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.018106451257811032')]
0.03283403230056999
Bad values at time t=84.0 lambda=2.9 rho=0.4 r=0.07
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.023220586428657465')]
0.020664554663772195
Bad values at time t=180.0 lambda=2.9 rho=0.4 r=0.07
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0061251710429375932')]
0.010137658532654387
Bad values at time t=365.0 lambda=2.9 rho=0.4 r=0.07
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0032165531061406349')]
0.42752080083158484
Bad values at time t=3.0 lambda=2.9 rho=0.4 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.25876836206781617')]
0.19729398234262313
Bad values at time t=10.0 lambda=2.9 rho=0.4 r=0.08
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.127825459005454')]
0.11471962413600867
Bad values at time 

0.010004286069626517
Bad values at time t=84.0 lambda=2.9 rho=0.4 r=0.14
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.018847268274131006')]
0.006531206307100918
Bad values at time t=180.0 lambda=2.9 rho=0.4 r=0.14
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0070490749005697123')]
0.003150667711258856
Bad values at time t=365.0 lambda=2.9 rho=0.4 r=0.14
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.0035721941166784984')]
0.30395429604739677
Bad values at time t=3.0 lambda=2.9 rho=0.4 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.23964924646506891')]
0.08815323923678627
Bad values at time t=10.0 lambda=2.9 rho=0.4 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.11711623742159931')]
0.044482934691397406
Bad values at time t=21.0 lambda=2.9 rho=0.4 r=0.15
any(probN>1=False, sum(probN)>1=False, (probS-sum(probN))=[mpf('0.059072702272879092')]
0.029246821852810877
Bad values at 

ZeroDivisionError: 

In [18]:
np.shape(result)

NameError: name 'result' is not defined

In [ ]:
result[0]

In [ ]:
def plotLikelihood(result):
    #Does some transformations to ensure that the graph is square, 
    # with r on x axis and rho on y, both increasing from bottom left
    plt.imshow((np.flip(result,axis=0)), cmap='coolwarm', interpolation='nearest',extent=[0.004,0.2,0.016,0.8],aspect=0.25)
#     plt.imshow((np.flip(np.transpose(result),axis=0)), cmap='coolwarm', interpolation='nearest',extent=[0.1,1,0.02,0.2],aspect=5)
    plt.colorbar()
    plt.show()

In [ ]:
plotLikelihood(result[0])

In [ ]:
scaledR = result[0]-max(max(i) for i in result[0])
expR = 10**(scaledR)
plotLikelihood(expR)

In [ ]:
allSum = sum([sum(r) for r in expR])
rhoNormalised = [sum(rho) for rho in expR]/allSum
rNormalised = [sum(rho) for rho in np.transpose(expR)]/allSum
rDistribution = np.cumsum(rNormalised)
print(np.where(rDistribution>0.025))
print(np.where(rDistribution<0.975))

In [ ]:
# rRange[8:14]

In [ ]:
rhoDistribution = np.cumsum(rhoNormalised)
print(np.where(rhoDistribution>0.025))
print(np.where(rhoDistribution<0.975))

In [ ]:
# rhoRange[13:28]

In [ ]:
pickle_out = open("pickled_grid_px20","wb")
pickle.dump(expR, pickle_out)
pickle_out.close()

In [ ]:
def plotLikelihood(result):
    #Does some transformations to ensure that the graph is square, 
    # with r on x axis and rho on y, both increasing from bottom left
    plt.imshow((np.flip(result,axis=0)), cmap='coolwarm', interpolation='nearest',extent=[0,0.2,0,0.8],aspect=0.25)
#     plt.imshow((np.flip(np.transpose(result),axis=0)), cmap='coolwarm', interpolation='nearest',extent=[0.1,1,0.02,0.2],aspect=5)
    # based on the constant slope of the plot of average clone size against time, r/rho=slope/lambda
    x = np.linspace(0,0.1,50)
    y = x/0.12
    plt.plot(x, y, '-r', label='y=2x+1')
    plt.colorbar()
    plt.show()

plotLikelihood(expR)

In [ ]:
allSum = sum([sum(r) for r in expR])
rhoNormalised = [sum(rho) for rho in expR]/allSum
rNormalised = [sum(rho) for rho in np.transpose(expR)]/allSum
rDistribution = np.cumsum(rNormalised)
print(np.where(rDistribution>0.025))
print(np.where(rDistribution<0.975))

In [ ]:
rRange[7:12]

In [ ]:
rhoDistribution = np.cumsum(rhoNormalised)
print(np.where(rhoDistribution>0.025))
print(np.where(rhoDistribution<0.975))

In [ ]:
rhoRange[37:42]